# Statistics Base Model for Demand Forecasting

The main objective is to optimize inventory and purchasing management, with a target of **reducing overstocking by 20%** within 6 months.

- Target Variable for Inventory Optimization: **Stock_Quantity**
- Target Variable for Demand Forecasting: **Sales_Volume**

### Metrics for models avaliation
- RMSE - Root Mean Squared Error
- MAE - Mean Absolute Error

## Import Libraries

In [1]:
# Standart Libraries 
import pandas as pd
import os
import plotly.express as px
import joblib

# Specialized Libraries
import logging
from sktime.forecasting.ets import AutoETS
from sktime.split import temporal_train_test_split
from sktime.transformations.series.impute import Imputer
from sktime.forecasting.compose import TransformedTargetForecaster, ColumnEnsembleForecaster
from sktime.transformations.compose import TransformerPipeline
from sktime.performance_metrics.forecasting import mean_absolute_error, mean_squared_error
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.statsforecast import StatsForecastADIDA
from sktime.transformations.series.boxcox import LogTransformer



# Notebook mlflow Loggings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")
logging.getLogger().setLevel(logging.WARNING)
pd.set_option('display.max_columns', None)

## Load Data

In [2]:
# Define data paths
data_path = os.path.join('../data', 'processed')
docs_path = os.path.join('../docs/')
path_models = os.path.join('../models/')

saved_models = os.listdir(path_models)

In [3]:
# Load Pickle file
# read_data = pd.read_pickle(data_path + '/data_for_train.pkl')
read_data = pd.read_pickle(data_path + '/feature_eng_complete.pkl')

# Time Series Base Model

In [4]:
# Create a copy of the original DataFrame to preserve the raw data
df = read_data.copy()

In [5]:
# Display the first five rows of the DataFrame to preview the data
df.head()


,received_date,lpo,in_season,product,product_id,category,sub_category,shelf_life_days,maximum_days_on_sale,unit_of_measurement,supplier_rating,supplier,supplier_id,distance_km,moq,storage_recommendation,temperature_classification,precipitation_classification,wind_classification,weather_severity,day_classification,is_holiday,is_weekend,sales_demand,sales_volume,lead_time,min_stock,max_stock,stock_quantity,delivery_lag,expiration_status,inventory_turnover_rate,doi_inventory_turnover,delivery_time_variation,Year,Month,Day,DayOfYear,Weekday,QuarterOfYear,WeekOfYear,avg_daily_sales,std_daily_sales,avg_daily_lag,std_daily_lag,std_demand_lead_time,safety_stock,rop,reorder_point_coverage,reorder_quantity,reorder_point_quantity_turnover,product_code,supplier_code,category_code,sub_category_code,supplier_rating_code,weather_severity_code,day_classification_code,sales_demand_code,expiration_status_code,in_season_code,is_holiday_code,is_weekend_code
0,2022-12-09,2022-12-07,False,Dijon Mustard,1070686|P,Pantry,Condiments,365,90,unit,3,Condiment Masters,1184993|S,75,60,Room Temperature,Warm,No precipitation,Gentle to Fresh Breeze,Moderate,Weekdays,False,False,Normal,50,4,240,300,317,2,Safe,0.839816,1212,2,2022,12,9,343,4,4,49,76.833333,31.801992,4.000000,3.162278,251.155485,411.894996,565.561662,7.360889,307,307.878826,57,11,5,6,2,0,3,1,2,0,0,0
1,2022-12-09,2022-12-07,True,Orange,1362741|P,Fresh Foods,Fruits,14,7,lb,3,OrchardBest Fruits,1677419|S,200,120,Room Temperature,Warm,No precipitation,Gentle to Fresh Breeze,Moderate,Weekdays,False,False,Normal,590,6,3678,4291,4608,2,Safe,0.378488,2689,4,2022,12,9,343,4,4,49,618.600000,390.070250,4.600000,2.701851,1869.056871,3065.253269,4302.453269,6.955146,3711,1562.671857,110,33,3,11,2,0,3,1,2,1,0,0
2,2022-12-09,2022-12-06,False,Asparagus,1308864|P,Fresh Foods,Vegetables,5,2,lb,5,Asparagus Experts,1197859|S,115,18,Refrigerated,Warm,No precipitation,Gentle to Fresh Breeze,Moderate,Weekdays,False,False,Normal,532,4,4652,5815,6765,3,Nearing,0.710163,1433,1,2022,12,9,343,4,4,49,986.111111,369.406092,5.666667,3.500000,3561.651790,5841.108936,8799.442269,8.923378,3944,6837.084898,7,1,3,26,4,0,3,1,1,0,0,0
3,2022-12-09,2022-11-30,False,Cashews,1393487|P,Pantry,Nuts & Seeds,180,60,lb,5,Nut & Seed Co.,1535281|S,125,55,Room Temperature,Warm,No precipitation,Gentle to Fresh Breeze,Moderate,Weekdays,False,False,Normal,15,4,72,90,80,9,Safe,0.725117,1403,-5,2022,12,9,343,4,4,49,19.250000,4.031129,5.750000,2.753785,53.884473,88.370536,261.620536,13.590677,77,254.608083,36,29,5,15,4,0,3,1,2,0,0,0
4,2022-12-10,2022-12-04,False,Frozen Shrimp,1345059|P,Frozen Foods,Seafood,180,60,lb,5,FrozenFoods Express,1579962|S,65,90,Frozen,Warm,No precipitation,Gentle to Fresh Breeze,Moderate,Saturday,False,True,High,14,5,40,90,52,6,Safe,0.693274,1468,-1,2022,12,10,344,5,4,49,10.909091,4.846742,5.727273,2.453198,29.167657,47.834958,113.289503,10.384871,54,85.521008,69,18,4,20,4,0,1,0,2,0,0,1


# Transform Data
## Stock Quantity

In [6]:
# Create a pivot table showing the average stock quantity per product for each received date.
# Missing dates are filled with 0 and the frequency is set to daily.
df_stock = df.pivot_table(index='received_date', columns='product', 
                          values='stock_quantity', aggfunc='mean', 
                          fill_value=0).asfreq('D', fill_value=0)


In [7]:
# Split the stock data into training and testing sets, reserving the last 28 days for testing
y_stock_train, y_stock_test = temporal_train_test_split(df_stock, test_size=28)


In [8]:
# Print the number of rows and columns in the training and testing datasets
print(f'Train shape: {y_stock_train.shape}')
print(f'Test shape: {y_stock_test.shape}')


Train shape: (643, 170)
Test shape: (28, 170)


In [9]:
# The target columns are the columns of the training DataFrame (the product_codes)
stock_target_columns = y_stock_train.columns.tolist()

# The forecast horizon (fh) will be the length of the test set (28 days)
stock_fh = list(range(1, len(y_stock_test) + 1))


## Sales Volume

In [10]:
# Create a pivot table showing the average sales volume per product for each received date.
# Fill missing values with 0 and set the frequency to daily.
df_sales = df.pivot_table(index='received_date', columns='product',
                          values='sales_volume', aggfunc='mean', 
                          fill_value=0 ).asfreq('D', fill_value=0)


In [11]:
# Split the sales data into training and testing sets, reserving the last 28 days for testing
y_sales_train, y_sales_test = temporal_train_test_split(df_sales, test_size=28)


In [12]:
# Print the dimensions of the training and testing datasets for sales data
print(f'Train shape: {y_sales_train.shape}')
print(f'Test shape: {y_sales_test.shape}')


Train shape: (643, 170)
Test shape: (28, 170)


In [13]:
# The target columns are the columns of the training DataFrame (the product_codes)
sales_target_columns = y_sales_train.columns.tolist()

# The forecast horizon (fh) will be the length of the test set (28 days)
sales_fh = list(range(1, len(y_sales_test) + 1))


In [14]:
def train_load(model: str, data_train: pd.Series, arch_name: str, path_models: str = path_models):

    # Define the filename for saving the trained forecaster
    MODEL_FILENAME = f'{arch_name}.joblib'
    full_path = path_models + MODEL_FILENAME

    if MODEL_FILENAME in saved_models:
        # Load the forecaster object
        print(f'Loading {MODEL_FILENAME}')
        model = joblib.load(full_path)
        print("Model loaded successfully!")
    else: 
        print('WAIT: Training....')
        model.fit(data_train);

        # Save the complete forecaster object
        print(f"Saving the trained forecaster to {MODEL_FILENAME}...")
        joblib.dump(model, full_path)
        print("Model saved successfully!")

    return model

# AutoETS pipeline for demand forecasting


## Estimator (AutoETS)


In [15]:
# Defining the base estimator: AutoETS
# sp=7 enforces the search for weekly seasonality
ets_estimator = AutoETS(sp=7, n_jobs=-1)


## Transformation and Modeling Pipeline


In [16]:
# Transformation pipeline (Imputation of NaN values if necessary)
transformer = Imputer(method="mean")


In [17]:
# Define the pipeline for a single time series (product)
# We use TransformedTargetForecaster to apply a Transformer (Imputer)
# followed by a Forecaster (AutoETS)
single_series_forecaster = TransformedTargetForecaster(
    steps=[
        ("imputer", transformer),  # Step 1: Imputation (Transformer)
        ("forecaster", ets_estimator)  # Step 2: Forecasting (Forecaster)
    ]
)


In [18]:
# We use ColumnEnsembleForecaster to apply the pipeline above to each column individually
fc_ets = ColumnEnsembleForecaster(
    forecasters=[
        ("ets_pipeline_" + str(col_name), single_series_forecaster, col_name)
        for col_name in sales_target_columns 
    ]
)


## Training and Forecasting - Sales Volume


In [19]:
# Load or train the baseline ensemble model using the sales training data
fc_ets = train_load(model=fc_ets, data_train=y_sales_train, arch_name='baseline_ets')

Loading baseline_ets.joblib
Model loaded successfully!


In [20]:
# Generate sales volume forecasts for the defined forecast horizon
y_sales_pred = fc_ets.predict(fh=sales_fh)


In [21]:
# Calculate the average across all columns (products) for each day
y_sales_test_mean = y_sales_test.mean(axis=1)
y_sales_pred_mean = y_sales_pred.mean(axis=1)


In [22]:
# Create a DataFrame to compare actual and predicted average daily sales
df_plot_mean = pd.DataFrame({
    'Real sales': y_sales_test_mean,
    'Predict sales': y_sales_pred_mean
})


In [23]:
# Reset the index and rename it to 'Date' for plotting
df_plot_mean = df_plot_mean.reset_index().rename(columns={'index': 'Date'})

# Create an interactive line plot comparing actual and predicted sales
fig = px.line(df_plot_mean, x='Date', y=['Real sales', 'Predict sales'], title='AutoETS - Real and Predict Sales Volume')
fig.show()


In [24]:
# Calculate RMSE (Root Mean Squared Error) across all products
rmse = mean_squared_error(y_sales_test, y_sales_pred, multioutput='uniform_average', square_root=True)
print(f'RMSE: {rmse}')

# Calculate MAE (Mean Absolute Error) across all products
mae = mean_absolute_error(y_sales_test, y_sales_pred, multioutput='uniform_average')
print(f'MAE: {mae}')


RMSE: 35.01296503732421
MAE: 12.81248082154992


In [25]:
# Calculate RMSE for each product individually
rmse_for_product = mean_squared_error(y_sales_test, y_sales_pred, multioutput='raw_values', square_root=True)
rmse_series = pd.Series(rmse_for_product, index=y_sales_test.columns)

# Display the Top 5 Products with the WORST RMSE (poorest performance)
print("\n--- Top 5 Products with WORST RMSE (Poorest Performance) ---")
print(rmse_series.sort_values(ascending=False).head(5))

# Display the Top 5 Products with the BEST RMSE (best performance)
print("\n--- Top 5 Products with BEST RMSE (Best Performance) ---")
print(rmse_series.sort_values(ascending=True).head(5))



--- Top 5 Products with WORST RMSE (Poorest Performance) ---
product
Zucchini      602.905977
Roast Beef    288.473194
Lettuce       287.048472
Asparagus     281.531259
Avocado       231.030360
dtype: float64

--- Top 5 Products with BEST RMSE (Best Performance) ---
product
Coconut Sugar           0.040430
Frozen Salmon Fillet    0.102647
Canned Sardines         0.115081
Cashews                 0.119738
Sugar                   0.127513
dtype: float64


**Conclusion:**  The AutoETS model was unable to adequately capture the seasonal structure previously identified in the sales data. As an alternative, AutoARIMA will be tested, as it may offer greater flexibility in modeling complex temporal patterns.

---

# AutoARIMA pipeline for demand forecasting


## Estimator (AutoARIMA)


In [26]:
# sp=7: Forces the search for weekly seasonality.
# suppress_warnings=True: Suppresses statistical warning logs during model fitting.
# stepwise=True: Optimizes parameter search for faster computation.
arima_estimator = AutoARIMA(
    sp=7, 
    suppress_warnings=True, 
    stepwise=True, 
    n_jobs=-1
)


## ARIMA Transformation and Modeling Pipeline


In [27]:
# Transformation pipeline (Imputation of NaN values if necessary)
transformer = Imputer(method="mean")

# Define the pipeline for a single time series (product)
# We use TransformedTargetForecaster to apply a Transformer (Imputer)
# followed by a Forecaster (AutoARIMA)
single_series_forecaster = TransformedTargetForecaster(
    steps=[
        ("imputer", transformer),  # Step 1: Imputation (Transformer)
        ("forecaster", arima_estimator)  # Step 2: Forecasting (Forecaster)
    ]
)

# We use ColumnEnsembleForecaster to apply the above pipeline to each column
fc_autoARIMA = ColumnEnsembleForecaster(
    forecasters=[
        ("arima_pipeline_" + str(col_name), single_series_forecaster, col_name)
        for col_name in sales_target_columns 
    ]
)


## Training and Forecasting - Sales Volume


In [28]:
# Load or train the baseline ensemble model using the sales training data
fc_autoARIMA = train_load(model=fc_autoARIMA, data_train=y_sales_train, arch_name='baseline_sarima_column_ensemble')

Loading baseline_sarima_column_ensemble.joblib
Model loaded successfully!


In [29]:
# Forecasting
y_sales_pred = fc_autoARIMA.predict(fh=sales_fh)

# Calculate the average sales across all products for each day
y_sales_test_mean = y_sales_test.mean(axis=1)
y_sales_pred_mean = y_sales_pred.mean(axis=1)

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sklearn/utils/d

In [30]:
# Create a DataFrame for plotting actual vs predicted sales
df_plot_mean = pd.DataFrame({
    'Real sales': y_sales_test_mean,
    'Predict sales': y_sales_pred_mean
})

In [31]:
# Reset the DataFrame index and rename the index column to 'Date' for clearer plotting
df_plot_mean = df_plot_mean.reset_index().rename(columns={'index': 'Date'})

# Plot the results
fig = px.line(df_plot_mean, x='Date', y=['Real sales', 'Predict sales'], title='AutoARIMA - Real and Predicted Sales Volume')
fig.show()

In [32]:
# Calculate overall RMSE and MAE
rmse = mean_squared_error(y_sales_test, y_sales_pred, multioutput='uniform_average', square_root=True)
print(f'RMSE: {rmse}')
mae = mean_absolute_error(y_sales_test, y_sales_pred, multioutput='uniform_average')
print(f'MAE: {mae}')

RMSE: 34.97194996172553
MAE: 12.869602730377917


In [33]:
# Calculate RMSE per product
rmse_for_product = mean_squared_error(y_sales_test, y_sales_pred, multioutput='raw_values', square_root=True)
rmse_series = pd.Series(rmse_for_product, index=y_sales_test.columns)

# Display top 5 products with worst RMSE (poorest performance)
print("\n--- Top 5 Products with WORST RMSE (Poorest Performance) ---")
print(rmse_series.sort_values(ascending=False).head(5))

# Display top 5 products with best RMSE (best performance)
print("\n--- Top 5 Products with BEST RMSE (Best Performance) ---")
print(rmse_series.sort_values(ascending=True).head(5))



--- Top 5 Products with WORST RMSE (Poorest Performance) ---
product
Zucchini      602.433317
Roast Beef    287.873171
Lettuce       286.394373
Asparagus     281.531175
Avocado       230.057041
dtype: float64

--- Top 5 Products with BEST RMSE (Best Performance) ---
product
Ricotta Cheese          0.000000
Coconut Sugar           0.040435
Frozen Salmon Fillet    0.099676
Canned Sardines         0.115086
Cashews                 0.119751
dtype: float64


**Conclusion:**  Neither AutoETS nor AutoARIMA were able to effectively capture the trends and seasonality in the data, likely due to the sparsity of the time series matrix.  
To address this, I will use the **ADIDA** model, which is more robust to this characteristic.

---

# ADIDA pipeline for demand forecasting

## Estimator (ADIDA)

In [34]:
# Base Estimator: ADIDA (Atemporal Demand Interval Demand Averaging)
# This model is specifically designed for intermittent demand.
intermittent_estimator = StatsForecastADIDA();

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/statsforecast/core.py:21: TqdmExperimentalWarning:

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sktime/forecasting/base/adapters/_generalised_statsforecast.py:413: UserWarning:

 ADIDA from statsforecast v1.4.0 does not support prediction of intervals in `predict_in_sample`. Consider upgrading to a newer version.

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sktime/forecasting/base/adapters/_generalised_statsforecast.py:426: UserWarning:

 ADIDA from statsforecast v1.4.0 does not support prediction of intervals in `predict`. Consider upgrading to a newer version.



## ADIDA Transformation and Modeling Pipeline


In [35]:
# Transformation pipeline (Imputation of NaN values if necessary)
transformer = Imputer(method="mean")

# Define the pipeline for a single time series (product)
# We use TransformedTargetForecaster to apply a Transformer (Imputer)
# followed by a Forecaster (ADIDA)
single_series_forecaster = TransformedTargetForecaster(
    steps=[
        ("imputer", transformer),  # Step 1: Imputation (Transformer)
        ("forecaster", intermittent_estimator)  # Step 2: Forecasting (Forecaster)
    ]
)

# Use ColumnEnsembleForecaster to apply the above pipeline to each column (product)
fc_ADIDA = ColumnEnsembleForecaster(
    forecasters=[
        ("ADIDA_pipeline_" + str(col_name), single_series_forecaster, col_name)
        for col_name in sales_target_columns 
    ]
)

## Training and Forecasting - Sales Volume


In [36]:
# Load or train the baseline ensemble model using the sales training data
fc_ADIDA = train_load(model=fc_ADIDA, data_train=y_sales_train, arch_name='baseline_ADIDA_column_ensemble')

Loading baseline_ADIDA_column_ensemble.joblib
Model loaded successfully!


In [37]:
# Forecasting
y_sales_pred = fc_ADIDA.predict(fh=sales_fh)

# Calculate the average sales across all products for each day
y_sales_test_mean = y_sales_test.mean(axis=1)
y_sales_pred_mean = y_sales_pred.mean(axis=1)

# Create a DataFrame for plotting actual vs predicted sales
df_plot_mean = pd.DataFrame({
    'Real sales': y_sales_test_mean,
    'Predict sales': y_sales_pred_mean
})

# Reset the DataFrame index and rename the index column to 'Date' for clearer plotting
df_plot_mean = df_plot_mean.reset_index().rename(columns={'index': 'Date'})

In [38]:
# Plot the results
fig = px.line(df_plot_mean, x='Date', y=['Real sales', 'Predict sales'], title='ADIDA - Real and Predicted Sales Volume')
fig.show()

In [39]:
# Calculate overall RMSE and MAE
rmse = mean_squared_error(y_sales_test, y_sales_pred, multioutput='uniform_average', square_root=True)
print(f'RMSE: {rmse}')
mae = mean_absolute_error(y_sales_test, y_sales_pred, multioutput='uniform_average')
print(f'MAE: {mae}')

RMSE: 35.06138108756066
MAE: 13.04937972691724


In [40]:
# Calculate RMSE per product
rmse_for_product = mean_squared_error(y_sales_test, y_sales_pred, multioutput='raw_values', square_root=True)
rmse_series = pd.Series(rmse_for_product, index=y_sales_test.columns)

# Display top 5 products with worst RMSE (poorest performance)
print("\n--- Top 5 Products with WORST RMSE (Poorest Performance) ---")
print(rmse_series.sort_values(ascending=False).head(5))

# Display top 5 products with best RMSE (best performance)
print("\n--- Top 5 Products with BEST RMSE (Best Performance) ---")
print(rmse_series.sort_values(ascending=True).head(5))



--- Top 5 Products with WORST RMSE (Poorest Performance) ---
product
Zucchini      602.888081
Roast Beef    288.742421
Lettuce       286.854278
Asparagus     281.684144
Avocado       230.532132
dtype: float64

--- Top 5 Products with BEST RMSE (Best Performance) ---
product
Coconut Sugar           0.033488
Cashews                 0.087401
Canned Sardines         0.114988
Almond Flour            0.116922
Frozen Salmon Fillet    0.120997
dtype: float64



**Conclusion:** The constant problem indicates that baseline models relying solely on autocorrelation have reached their limit. Transitioning to XGBoost is now the most logical and promising step, as it allows the inclusion of temporal variables — such as day of the week and month — as predictors, breaking free from the naive mean cycle and enabling more robust forecasting.

---

# AutoARIMA Aggregate Sales Volume Forecasting
## Prepare Data for Aggregate Format

In [41]:
# Create a new DataFrame that shows the total sales volume for each received date
sales_df = pd.DataFrame(df.groupby('received_date')['sales_volume'].sum())


In [42]:
# Create and display a box plot of aggregated sales volume with customized layout settings
fig = px.box(sales_df, title='Aggregated Sales Volume')
fig.update_layout(width=400, height=600, xaxis_title=None, yaxis_title=None, title_x=0.5)
fig.show()


In [43]:
# Define the upper threshold for sales volume to identify outliers
upper_fence = 3386

# Extract rows where sales volume exceeds the upper threshold (outliers)
outliers = sales_df[sales_df['sales_volume'] > upper_fence]

# Create a filtered DataFrame excluding the outliers
sales_df_filtered = sales_df[sales_df['sales_volume'] <= upper_fence]


In [44]:
# Convert the DataFrame to a daily frequency, filling missing dates with zero sales volume
sales_df_filtered = sales_df_filtered.asfreq('D', fill_value=0)


In [45]:
# Split the time series data into training and testing sets, reserving the last 28 days for testing
y_train, y_test = temporal_train_test_split(sales_df_filtered, test_size=28)


In [46]:
# Initialize an AutoARIMA model with weekly seasonality and differencing to handle trends and seasonality
fc_agg = AutoARIMA(seasonal=True, sp=7, d=1, D=1, suppress_warnings=True)


## Training and Forecasting - Aggregate Sales Volume

In [47]:
# Load or train the baseline ensemble model using the sales training data
fc_agg = train_load(model=fc_agg, data_train=y_train, arch_name='baseline_aggregation')

Loading baseline_aggregation.joblib
Model loaded successfully!


In [48]:
# Define the filename for saving the trained forecaster
MODEL_FILENAME = 'baseline_aggregation.joblib'

if MODEL_FILENAME in saved_models:
    # Load the forecaster object
    fc_agg = joblib.load(path_models + MODEL_FILENAME)
    print("Model loaded successfully!")
else: 
    print('WAIT: Training')
    fc_agg.fit(y_train);

    # Save the complete forecaster object
    print(f"Saving the trained forecaster to {MODEL_FILENAME}...")
    joblib.dump(fc_agg, path_models + MODEL_FILENAME)
    print("Model saved successfully!")


Model loaded successfully!


In [49]:
# The forecast horizon (fh) will be the length of the test set (28 days)
fh = list(range(1, len(y_test) + 1))

# Forecasting
y_pred = fc_agg.predict(fh=fh)


/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [50]:
# Create a DataFrame for plotting actual vs predicted sales
df_plot = pd.DataFrame({
    'Real sales': y_test['sales_volume'],
    'Predict sales': y_pred['sales_volume']
})

# Reset the DataFrame index and rename the index column to 'Date' for clearer plotting
df_plot = df_plot.reset_index().rename(columns={'index': 'Date'})

In [51]:
# Plot the results
fig = px.line(df_plot, x='Date', y=['Real sales', 'Predict sales'], title='AutoARIMA Aggregated - Real and Predicted Sales Volume')
fig.show()

In [52]:
# Calculate overall RMSE and MAE
rmse = mean_squared_error(y_test, y_pred, multioutput='uniform_average', square_root=True)
print(f'RMSE: {rmse}')
mae = mean_absolute_error(y_test, y_pred, multioutput='uniform_average')
print(f'MAE: {mae}')

RMSE: 912.2274293061655
MAE: 663.8803270045134


**Finding:** The RMSE of 35 was misleading; the 912 is reliable.

**Confirmations:** AutoARIMA captured seasonality in the aggregated data, but RMSE increased due to higher variability.

**Reason for the increase:** Previously, the model predicted constant and irrelevant values.

**Conclusion:** We now have a real metric to assess demand.


**Strategy to Return to Product-Level Forecasting**  
**Goal:** Transfer the forecasting power of the aggregated model to each individual product.

**Recommended Actions:**
- Reapply AutoARIMA in a multi-series format.  
- **Outlier Treatment:** Remove outliers from each individual series.  
- **Log or Box-Cox Transformations:** Stabilize variance and improve SARIMA performance.

---

# AutoARIMA pipeline for demand forecasting - Log Transform


## ARIMA Transformation and Modeling Pipeline


In [53]:
transformer = TransformerPipeline(
    [
        ("imputer", Imputer(method="mean")),
        ("log", LogTransformer(offset=1.0))  # Log com offset automático
    ]
)

# Pipeline de Série Única
single_series_forecaster = TransformedTargetForecaster(
    steps=[
        ("transformer", transformer), # Aplica Imputação e Log
        ("fc_autoarima", AutoARIMA(sp=7, suppress_warnings=True, stepwise=True, n_jobs=-1))
    ]
)

# We use ColumnEnsembleForecaster to apply the above pipeline to each column
fc_arima = ColumnEnsembleForecaster(
    forecasters=[
        ("arima_pipeline_" + str(col_name), single_series_forecaster, col_name)
        for col_name in sales_target_columns 
    ]
)


## Training and Forecasting - Sales Volume


In [54]:
# Load or train the baseline ensemble model using the sales training data
fc_log_transformed = train_load(model=fc_arima, data_train=y_sales_train, arch_name='baseline_log_transform')

Loading baseline_log_transform.joblib
Model loaded successfully!


In [55]:
# Forecasting
y_sales_pred = fc_log_transformed.predict(fh=sales_fh)

# Calculate the average sales across all products for each day
y_sales_test_mean = y_sales_test.mean(axis=1)
y_sales_pred_mean = y_sales_pred.mean(axis=1)

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/rb/Projects/portfolio/smart-supply-chain-ai/.venv/lib/python3.11/site-packages/sklearn/utils/d

In [56]:
# Create a DataFrame for plotting actual vs predicted sales
df_plot_mean = pd.DataFrame({
    'Real sales': y_sales_test_mean,
    'Predict sales': y_sales_pred_mean
})

In [57]:
# Reset the DataFrame index and rename the index column to 'Date' for clearer plotting
df_plot_mean = df_plot_mean.reset_index().rename(columns={'index': 'Date'})

# Plot the results
fig = px.line(df_plot_mean, x='Date', y=['Real sales', 'Predict sales'], title='AutoARIMA Data Log - Real and Predicted Sales Volume')
fig.show()

In [58]:
# Calculate overall RMSE and MAE
rmse = mean_squared_error(y_sales_test, y_sales_pred, multioutput='uniform_average', square_root=True)
print(f'RMSE: {rmse}')
mae = mean_absolute_error(y_sales_test, y_sales_pred, multioutput='uniform_average')
print(f'MAE: {mae}')

RMSE: 32.85037468746352
MAE: 7.1384863864433274


In [59]:
# Calculate RMSE per product
rmse_for_product = mean_squared_error(y_sales_test, y_sales_pred, multioutput='raw_values', square_root=True)
rmse_series = pd.Series(rmse_for_product, index=y_sales_test.columns)

# Display top 5 products with worst RMSE (poorest performance)
print("\n--- Top 5 Products with WORST RMSE (Poorest Performance) ---")
print(rmse_series.sort_values(ascending=False).head(5))

# Display top 5 products with best RMSE (best performance)
print("\n--- Top 5 Products with BEST RMSE (Best Performance) ---")
print(rmse_series.sort_values(ascending=True).head(5))



--- Top 5 Products with WORST RMSE (Poorest Performance) ---
product
Zucchini      608.078440
Roast Beef    295.926651
Lettuce       291.361588
Asparagus     283.456752
Avocado       236.349919
dtype: float64

--- Top 5 Products with BEST RMSE (Best Performance) ---
product
Coconut Sugar           0.010527
Halibut                 0.017649
Cashews                 0.018795
Canned Sardines         0.023927
Frozen Salmon Fillet    0.026899
dtype: float64


---

**Definitive Conclusion: Failure in Univariate Modeling**  
The fact that the forecast remains a constant value (≈0.0919) even after Logarithmic Transformation and the use of AutoARIMA allows us to draw the following conclusions:

- **Ignored Model**: AutoARIMA failed to detect significant correlations (Trend, Seasonality, or AR/MA), even in the transformed data.

- **Constant Log-Value**: The value of 0.0919 is the Arithmetic Mean of the transformed sales for most of the 170 products. When the model finds no signal, it predicts the mean.

- **Misleading Metrics**: The RMSE of 32.85 and MAE of 7.13 are low only because the original series contains many zeros. The model predicts a low value (the mean), and since the actual value is often zero, the absolute error is small. This is the same issue that made the initial RMSE of 35 deceptive. The model is accurate in predicting the "near-zero" state but useless for determining when a sales event will occur.

- **Extreme Intermittency Problem**: The abundance of zeros completely drowns out the signal of seasonality and trend, rendering them invisible to any model based on temporal correlation (ARIMA or ETS).

---
